In [7]:
import tensorflow as tf
import numpy as np

In [8]:
#Artificial dataset, lets say XOR
def xor():
    X = np.array([[0,1], [1,0], [1,1],[0,0]], dtype=float)
    Y = np.array([1,1,0,0], dtype=float).reshape((-1,1))
    return X,Y

def square_function():
    X = []
    Y = []
    for x in range(10):
        X.append(x)
        Y.append(x**2)
    return np.array(X,dtype=np.float32).reshape(-1,1),np.array(Y, dtype=np.float32).reshape(-1,1)



# Simple autoencoder

So basically the only thing that is changing is the loss function, we want to get the same output as the input. This can be used to reduce the dimensionality of the input (encoding) with hidden layers, then we get the input back by decoding later, this is what the network is training.

In [9]:
#input

X,Y = square_function()

batch_size = 10

#Helper functions
def fully_connected(x, size, activation=tf.nn.relu):
    b = tf.Variable(tf.zeros([size]))
    weights = tf.Variable(tf.truncated_normal([x.get_shape()[1].value, size]))
    return activation(b + tf.matmul(x, weights))



INPUT_SHAPE = X.shape

x = tf.placeholder("float", [None,1])
y = tf.placeholder("float", [None,1])

#simple neural network to solve the xor problem

h1 = fully_connected(x, 20, activation=tf.nn.tanh)
print(h1.get_shape())

h2 = fully_connected(h1, 20)
print(h2.get_shape())

h3 = fully_connected(h2, 20)
print(h2.get_shape())

out = fully_connected(tf.nn.dropout(h2, 0.5), 1)
print(out.get_shape())

loss = tf.reduce_mean((out-x)**2)
tf.scalar_summary("loss", loss)


optimizer = tf.train.AdamOptimizer(0.1)
train_step = optimizer.minimize(loss)



(?, 20)
(?, 20)
(?, 20)
(?, 1)


# Run the session

In [10]:
init =  tf.initialize_all_variables()

num_epochs=1000
#merge = tf.merge_all_summaries()

with tf.Session() as sess:
    
    writer = tf.train.SummaryWriter("./summary", sess.graph)
    sess.run(init)
    #sess.run(merge)

    for epoch in range(num_epochs):
        for batch in range(X.shape[0]//batch_size):
            feed_dict = {
                x : X[batch*batch_size:batch*batch_size + batch_size],
                y : Y[batch*batch_size:batch*batch_size+batch_size]
            }
            _, lossVal = sess.run([train_step, loss], feed_dict=feed_dict)
        #writer.add_summary(merged, epoch)
        if(epoch%40 == 0):
            print("%d. Epoch Loss: %.4f "  %(epoch, lossVal))
        correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        

0. Epoch Loss: 16.9888 
40. Epoch Loss: 6.8400 
80. Epoch Loss: 6.1811 
120. Epoch Loss: 6.0960 
160. Epoch Loss: 2.6659 
200. Epoch Loss: 0.5074 
240. Epoch Loss: 0.6058 
280. Epoch Loss: 1.7008 
320. Epoch Loss: 1.4746 
360. Epoch Loss: 2.0308 
400. Epoch Loss: 1.4831 
440. Epoch Loss: 1.9276 
480. Epoch Loss: 3.1667 
520. Epoch Loss: 0.9435 
560. Epoch Loss: 1.9251 
600. Epoch Loss: 0.8479 
640. Epoch Loss: 1.7329 
680. Epoch Loss: 0.7715 
720. Epoch Loss: 1.4999 
760. Epoch Loss: 1.4522 
800. Epoch Loss: 2.0901 
840. Epoch Loss: 0.6398 
880. Epoch Loss: 2.6346 
920. Epoch Loss: 1.6160 
960. Epoch Loss: 3.0815 
